In [1]:
import chess

PIECE_VALUE = {
    chess.PAWN: 1,
    chess.KNIGHT: 3,
    chess.BISHOP: 3,
    chess.ROOK: 5,
    chess.QUEEN: 9,
    chess.KING: 0  # Il valore del re è gestito separatamente
}

# Tabelle di posizione per il pedone
PAWN_TABLE = [
    0, 0, 0, 0, 0, 0, 0, 0,
    5, 10, 10, -20, -20, 10, 10, 5,
    5, -5, -10, 0, 0, -10, -5, 5,
    0, 0, 0, 20, 20, 0, 0, 0,
    5, 5, 10, 25, 25, 10, 5, 5,
    10, 10, 20, 30, 30, 20, 10, 10,
    50, 50, 50, 50, 50, 50, 50, 50,
    0, 0, 0, 0, 0, 0, 0, 0
]

# Tabelle di posizione per il cavallo
KNIGHT_TABLE = [
    -50, -40, -30, -30, -30, -30, -40, -50,
    -40, -20, 0, 5, 5, 0, -20, -40,
    -30, 5, 10, 15, 15, 10, 5, -30,
    -30, 0, 15, 20, 20, 15, 0, -30,
    -30, 5, 15, 20, 20, 15, 5, -30,
    -30, 0, 10, 15, 15, 10, 0, -30,
    -40, -20, 0, 0, 0, 0, -20, -40,
    -50, -40, -30, -30, -30, -30, -40, -50,
]

# Tabelle di posizione per l'alfiere
BISHOP_TABLE = [
    -20, -10, -10, -10, -10, -10, -10, -20,
    -10, 5, 0, 0, 0, 0, 5, -10,
    -10, 10, 10, 10, 10, 10, 10, -10,
    -10, 0, 10, 10, 10, 10, 0, -10,
    -10, 5, 5, 10, 10, 5, 5, -10,
    -10, 0, 5, 10, 10, 5, 0, -10,
    -10, 0, 0, 0, 0, 0, 0, -10,
    -20, -10, -10, -10, -10, -10, -10, -20
]

# Tabelle di posizione per la torre
ROOK_TABLE = [
    0, 0, 0, 5, 5, 0, 0, 0,
    -5, 0, 0, 0, 0, 0, 0, -5,
    -5, 0, 0, 0, 0, 0, 0, -5,
    -5, 0, 0, 0, 0, 0, 0, -5,
    -5, 0, 0, 0, 0, 0, 0, -5,
    -5, 0, 0, 0, 0, 0, 0, -5,
    5, 10, 10, 10, 10, 10, 10, 5,
    0, 0, 0, 0, 0, 0, 0, 0
]

# Tabelle di posizione per la regina
QUEEEN_TABLE = [
    -20, -10, -10, -5, -5, -10, -10, -20,
    -10, 0, 5, 0, 0, 0, 0, -10,
    -10, 5, 5, 5, 5, 5, 0, -10,
    0, 0, 5, 5, 5, 5, 0, -5,
    -5, 0, 5, 5, 5, 5, 0, -5,
    -10, 0, 5, 5, 5, 5, 0, -10,
    -10, 0, 0, 0, 0, 0, 0, -10,
    -20, -10, -10, -5, -5, -10, -10, -20
]

# Tabelle di posizione per il re (inizio gioco)
KING_INITGAME_TABLE = [
    20, 30, 10, 0, 0, 10, 30, 20,
    20, 20, 0, 0, 0, 0, 20, 20,
    -10, -20, -20, -20, -20, -20, -20, -10,
    -20, -30, -30, -40, -40, -30, -30, -20,
    -30, -40, -40, -50, -50, -40, -40, -30,
    -30, -40, -40, -50, -50, -40, -40, -30,
    -30, -40, -40, -50, -50, -40, -40, -30,
    -30, -40, -40, -50, -50, -40, -40, -30
]

# Tabelle di posizione per il re (fine gioco)
KING_ENDGAME_TABLE = [
    -50, -40, -30, -20, -20, -30, -40, -50,
    -30, -20, -10, 0, 0, -10, -20, -30,
    -30, -10, 20, 30, 30, 20, -10, -30,
    -30, -10, 30, 40, 40, 30, -10, -30,
    -30, -10, 30, 40, 40, 30, -10, -30,
    -30, -10, 20, 30, 30, 20, -10, -30,
    -30, -30, 0, 0, 0, 0, -30, -30,
    -50, -30, -30, -30, -30, -30, -30, -50
]

In [2]:
from chessgame import StateChessGame
import numpy as np

class EvaluateBoardWithoutKing:
    def __init__(self, evaluate_end_game_phase=False, normalize_result=False):
        self.evaluate_end_game_phase = evaluate_end_game_phase
        self.normalize_result = normalize_result
        self.h_max_value = 9999
        self.h_min_value = -9999

    def __h(self, state: StateChessGame):
        if self.evaluate_end_game_phase:
            return self.__h(state.game_board)
        elif self.normalize_result:
            raw_eval = self.__h(state.game_board)
            return self.__normalize(raw_eval)
        else:
            return self.__h(state.game_board)

    def h(self, board):
        if self.evaluate_end_game_phase:
            game_over_eval = None
            if board.is_checkmate():
                outcome = board.outcome()
                if outcome is not None:
                    if outcome.winner:
                        game_over_eval = np.inf
                    else:
                        game_over_eval = -np.inf
            if board.is_stalemate() or board.is_insufficient_material() or board.is_seventyfive_moves() or board.is_fivefold_repetition():
                game_over_eval = 0

            if game_over_eval is not None:
                return game_over_eval
        # Se la partita è finita, restituisce un valore molto alto o molto basso
        if board.is_game_over():
            if board.is_checkmate():
                return -99 if board.turn else 99
            else:
                return 0  # Gestisce stallo e materiale insufficiente

        # Valutazione basata sui soli pezzi, ottimizzata
        eval = sum(PIECE_VALUE[piece] * (len(board.pieces(piece, chess.WHITE)) - len(board.pieces(piece, chess.BLACK)))
                   for piece in PIECE_VALUE)

        # Aggiungi qui altre euristiche, come la posizione dei pezzi, la mobilità, ecc.

        # Preferisce chi ha il turno di gioco, dato che potrebbe avere l'iniziativa
        eval += 0.1 if board.turn else -0.1

        return eval

    def __normalize(self, value):
        # Normalizza il valore in un range da -100 a +100
        if value >= 0:
            # Normalizzazione per valori positivi
            normalized = (value / self.h_max_value) * 100
        else:
            # Normalizzazione per valori negativi
            normalized = (value / abs(self.h_min_value)) * 100

        # Limita il valore normalizzato tra -100 e +100
        normalized = max(min(normalized, 100), -100)
        return normalized

In [3]:
from random import choice
from itertools import islice


def game_run(min, max):
    board = chess.Board(fen="rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1")
    moves = board.legal_moves
    #evaluation1 = EvaluatePiecePositions1()
    evaluation = EvaluateBoardWithoutKing()
    # evaluation2 = EvaluateMobility2()
    #print(board)
    #print(evaluation.h(board))
    #print(evaluation1.h(board))
    # print(evaluation2.h(board))
    move = next(islice(moves, 10, 11), None)
    board.push(move)
    moves = board.legal_moves
    #print(board)
    #print(evaluation.h(board))
    # print(evaluation1.h(board))
    # print(evaluation2.h(board))
    for i in range(700):
        moves = board.legal_moves
        if board.turn:
            # k = moves.count()
            # if k == 0: break
            # r = randrange(0, k)
            # if r == 0:
            #     move = next(islice(moves, 0, 1), None)
            # else:
            #     move = next(islice(moves, r - 1, r), None)
            move = choice(list(moves))
        else:
            # k = moves.count()
            # if k == 0: break
            # r = randrange(0, k)
            # if r == 0:
            #     move = next(islice(moves, 0, 1), None)
            # else:
            #     move = next(islice(moves, r - 1, r), None)
            move = choice(list(moves))
        board.push(move)
        #print(board.turn)
        #print(board)
        #print(evaluation.h(board))
        if min > evaluation.h(board):
            min = evaluation.h(board)
        if max < evaluation.h(board):
            max = evaluation.h(board)
        if board.is_game_over():
            return min, max
    #print(evaluation1.h(board))
    #print(evaluation1.h(board))
    #print(evaluation2.h(board))
    return min, max


min = float("inf")
max = float("-inf")
for p in range(16000):
    min1, max1 = game_run(min, max)
    print(min1, max1)
    if min1 < min: min = min1
    if max1 > max: max = max1
print("end")
print(min, max)
# board = chess.Board(fen="rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1")
# evaluation = EvaluateKingSafety()
# %timeit evaluation.h(board)
#%timeit evaluation1.h(board)
# %timeit game_run(min, max)

-9.1 20.1
-9.1 28.9
-18.1 28.9
-18.1 28.9
-18.1 99
-18.1 99
-18.1 99
-18.1 99
-18.1 99
-18.1 99
-18.1 99
-18.1 99
-18.1 99
-18.1 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99
-99 99


KeyboardInterrupt: 

In [ ]:
# import chess
# from chessgame.StateChessGame import StateChessGame
# 
# board = chess.Board(fen="rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1")
# state = StateChessGame(game_board=board, state_parent=None, move=None)
# print(state.game_board.turn)
# print(state.turn())
# 
# %timeit state.game_board.turn
# %timeit state.turn()